In [1]:
import cv2
import time
import numpy as np

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))


cap = cv2.VideoCapture(0)
time.sleep(3)

#using for loop to give webcam time to calibrate
for i in range(30):
    ret, background = cap.read()
    
background = np.flip(background, axis=1) #store the background, laterally flip
print("captured")


while (cap.isOpened()): #while camera is open
    ret, frame = cap.read()
    if not ret: #if return value is false, break
        break
    
    frame = np.flip(frame, axis=1) #laterally flip the frame

   
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    cv2.imshow('hsv',hsv)
   
    lower_red = np.array([0, 120, 50])
    upper_red = np.array([10, 255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    mask1 = mask1 + mask2
#    cv2.imshow('mask1',mask1)
#    res = cv2.bitwise_and(hsv,hsv, mask= mask1)
#    bilateral = cv2.bilateralFilter(res,15,75,75)
#    cv2.imshow('bilateral',bilateral)
    kernel = np.ones((3,3),np.uint8)
    #to reduce noise
    opening = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, kernel)

    
    mask_inv = cv2.bitwise_not(mask1)
#    cv2.imshow('mask_inv',mask_inv)
    res1 = cv2.bitwise_and(frame, frame, mask=mask_inv)
#    cv2.imshow('bckframe',res1)
    res2 = cv2.bitwise_and(background, background, mask=mask1)
#   cv2.imshow('redreplaced',res2)
    Output = cv2.addWeighted(res1, 1, res2, 1, 0)
    out.write(Output)    #saving the output video in "output.avi"
    cv2.imshow("INVISIBLE",Output)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break


cap.release()
out.release()
cv2.destroyAllWindows()

captured


In [11]:
import numpy as np

# Load two images
img1 = cv2.imread(r'C:\Users\pavni\OneDrive\Desktop\Tinkerer course\Opencv-python-learn\gyanam.png')
img2 = cv2.imread(r'C:\Users\pavni\OneDrive\Desktop\Tinkerer course\Opencv-python-learn\earth.png')

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]

# Now create a mask of logo and create its inverse mask
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# add a threshold
ret, mask = cv2.threshold(img2gray, 190, 255, cv2.THRESH_BINARY_INV) #As the colors in the image are not very bright therefore 
# a threshold of 220 will capture all of them

mask_inv = cv2.bitwise_not(mask)
cv2.imshow("m", mask)
cv2.imshow("i", mask_inv)
# # Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
# #Mask is binary, the binary operation is operated only when mask is 1 i.e white
cv2.imshow("b", img1_bg)
# # Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
cv2.imshow("f", img2_fg)
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst

cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()